In [ ]:
#Import necessary libraries and add script path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append(r"C:\Users\marku\Documents\SteelProdData_Project\src\scripts")

In [ ]:
# Load the normalized training and testing data
train_data_orig = pd.read_csv('C:\\Users\\marku\\Documents\\SteelProdData_Project\\data\\raw\\normalized_train_data.csv')
test_data_orig = pd.read_csv('C:\\Users\\marku\\Documents\\SteelProdData_Project\\data\\raw\\normalized_test_data.csv')


In [ ]:
#Concat training and testing data for Data Exploration
conc_data=pd.concat([train_data_orig, test_data_orig], ignore_index=True)

In [ ]:
#Check for missing values

# Define a set of values that should be treated as empty/missing
EMPTY_TOKENS = {"", " ", "NA", "N/A", "null", "None", "nan", "-"}

# Create a copy of the dataframe to avoid modifying the original
conc_data_check = conc_data.copy()

# Convert empty strings and placeholder values to NaN in object columns
obj_cols = conc_data_check.select_dtypes(include=["object"]).columns
for c in obj_cols:
    conc_data_check[c] = (
        conc_data_check[c]
        .astype(str)        # Convert all values to strings
        .str.strip()        # Remove leading/trailing whitespace
        .replace(EMPTY_TOKENS, np.nan)  # Replace empty tokens with NaN
    )

# Create a summary dataframe with statistics about each column
summary = pd.DataFrame({
    "dtype": conc_data_check.dtypes.astype(str),      # Data type of each column
    "rows_total": len(conc_data_check),               # Total number of rows
    "non_null": conc_data_check.notna().sum(),        # Count of non-null values
    "nulls": conc_data_check.isna().sum(),            # Count of null values
    "null_frac": (conc_data_check.isna().mean()),     # n_NaN/n_all_Entries
    "n_unique": conc_data_check.nunique(dropna=True), # Count of unique non-null values
})
# Sort the summary by null fraction
summary = summary.sort_values("null_frac", ascending=False)


display(summary)

In [ ]:
#Replace missing values with mean of the columns -Not necessary
numerical_cols = conc_data.select_dtypes(include=np.number).columns.tolist() 
conc_data[numerical_cols] = conc_data[numerical_cols].apply(lambda x: x.fillna(x.mean()), axis=0) # Fill NaN with column mean

In [ ]:
#Check for duplicates

# Count duplicate rows based on all input and output columns
n_dup = conc_data_check.duplicated().sum()
print(f"Duplikate über input1..input21 + output: {n_dup}")

# Display duplicate rows
dup_rows = conc_data_check[conc_data_check.duplicated(keep=False)].copy()

display(dup_rows.head(5))  # Display first 5 duplicate rows


In [ ]:
plt.hist(train_data_orig["output"])
plt.hist(test_data_orig["output"])
plt.xlabel('Output')
plt.ylabel('n') 
plt.title('Distribution of Output')
plt.show()

In [ ]:
import seaborn as sns

g=sns.pairplot(conc_data.sample(n=300, random_state=42)) #Plot pairwise relationships with sample of 300 rows

In [ ]:
#Heatmap

num_cols = conc_data.select_dtypes(include=["number"]).columns.tolist()
num_cols.remove("output")                             #Remove column output from correlation analysis

# Plot correlation heatmap
corr_cols = num_cols[:]             #List of numerical columns for correlation
corr = conc_data[corr_cols].corr()  
plt.figure(figsize=(10, 8))
plt.imshow(corr.values, aspect="auto")
plt.title("Korrelation")
plt.colorbar()

plt.xticks(range(len(corr_cols)), corr_cols, rotation=90, fontsize=8) #Rotate x-axis labels for better readability
plt.yticks(range(len(corr_cols)), corr_cols, fontsize=8) 

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X=conc_data.drop("output" , axis=1 ).values
y=conc_data["output"].values

# Standardize the inputs 
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reduce the 21 features down to 2
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

#For plotting cumulative variance
pca_2=PCA()  
pca_2.fit(X_scaled)

# Create the scatter plot
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis', alpha=0.5) #Y-values as color
plt.colorbar(label='Output Value')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA Analysis: Features reduced to 2D')
plt.show()

# Plot cumulative explained variance
cumulative_variance=np.cumsum(pca_2.explained_variance_ratio_)

plt.plot(range(1,22),cumulative_variance, marker='o', linestyle='--')

#Add horizontal lines for 90% and 95% variance
plt.axhline(y=0.90, color='r', linestyle='-', label='90% Variance')
plt.axhline(y=0.95, color='g', linestyle='-', label='95% Variance')

plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Variance: How much info do we keep?')
plt.grid(True)
plt.show()

In [ ]:
#Test/Train_Split
X=conc_data.drop("output" , axis=1 ).values
y=conc_data["output"].values


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=35)


In [ ]:
#Establish Baseline Model with Linear Regression

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
#s=StandardScaler()       #Data is standardized using Min-Max Scaling
#s.fit(X_train) 

#X_train_trans= s.transform(X_train) 
#X_test_trans=s.transform(X_test)

model_lr=LinearRegression()

model_lr.fit(X_train, y_train)

print(model_lr.score(X_test,y_test))

In [ ]:
#RandomForestRegressor default parameters

from sklearn.ensemble import RandomForestRegressor

model_rfreg=RandomForestRegressor()

model_rfreg.fit(X_train, y_train)

print(model_rfreg.score(X_test,y_test))

In [ ]:
#Standard Gaussian Process (RBF and COnstant Kernel)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# Define the kernel (covariance function)
# Here we use a constant kernel multiplied by an RBF kernel
kernel = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))
model_gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=0.1)
model_gp.fit(X_train, y_train)

from plot_helper import evaluate_model, plot_data
evaluate_model(model_gp, X_test, y_test)
plot_data(X_test, y_test, model_gp.predict(X_test), title="Model Predictions vs Ground Truth on Test Data")


#Terminated due to runtime issues

In [ ]:
# Add the specific gpytorch site-packages path to sys.path and try import
import sys
sys.path.append(r'c:\users\marku\appdata\local\packages\pythonsoftwarefoundation.python.3.13_qbz5n2kfra8p0\localcache\local-packages\python313\site-packages')

#sys.path.append(r'C:\Users\marku\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\Scripts')

In [ ]:
#Mixed Gaussian Process (COnstant Kernel, Matern, DotProduct, Noise)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, DotProduct, WhiteKernel, ConstantKernel

# Define the kernel (covariance function)

kernel = (ConstantKernel(1.0, (1e-3, 1e3)) * Matern(length_scale=[1.0]*21, nu=1.5) + 
    DotProduct(sigma_0=1.0) + 
    WhiteKernel(noise_level=0.1)
)
model_gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=0.1)
model_gp.fit(X_train, y_train)

from plot_helper import evaluate_model, plot_data
evaluate_model(model_gp, X_test, y_test)
plot_data(X_test, y_test, model_gp.predict(X_test), title="Model Predictions vs Ground Truth on Test Data")

In [ ]:
import torch

# 1. Check if CUDA is reachable
print(f"CUDA is available: {torch.cuda.is_available()}")

# 2. Check the version PyTorch is using
print(f"PyTorch CUDA version: {torch.version.cuda}")

# 3. Check the GPU name
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# GPU-accelerated Gaussian Process using GPyTorch (uses CUDA if available)
import torch
import gpytorch
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

s=StandardScaler()       
s.fit(X_train) 

X_train_trans= s.transform(X_train) 
X_test_trans=s.transform(X_test)

# Select device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Convert numpy data to torch tensors on the selected device
X_train_t = torch.from_numpy(X_train_trans.astype(np.float32)).to(device)
y_train_t = torch.from_numpy(y_train.astype(np.float32)).to(device)
X_test_t = torch.from_numpy(X_test_trans.astype(np.float32)).to(device)
y_test_t = torch.from_numpy(y_test.astype(np.float32)).to(device)

# Define the GP model
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=21)
        ) + gpytorch.kernels.LinearKernel()
        #self.covar_module[0].outputscale = 1.2 #Manually set the initial weight, since preds are offset in plot_data

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    
# Initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)
model_gt = ExactGPModel(X_train_t, y_train_t, likelihood).to(device)

# Train the model 
model_gt.train()
likelihood.train()

#Use the Adam optimizer from torch
optimizer = torch.optim.Adam(model_gt.parameters(), lr=0.1) 
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model_gt)

# Training loop
training_iter = 100
for i in range(training_iter):
    optimizer.zero_grad()
    output = model_gt(X_train_t)
    loss = -mll(output, y_train_t)
    loss.backward()
    print(f'Iter {i+1}/{training_iter} - Loss: {loss.item():.4f}')
    optimizer.step()

# Evaluation 
model_gt.eval()
likelihood.eval()


with torch.no_grad():
    preds = likelihood(model_gt(X_test_t)) #equal to scikit learn .predict()
    mean = preds.mean.cpu().numpy()        #convert to numpy on CPU
    
# Metrics and plotting
r2 = r2_score(y_test, mean)
mae = mean_absolute_error(y_test, mean)
mse = mean_squared_error(y_test, mean)
print(f'GPyTorch GPU GP — R2: {r2:.4f}, MAE: {mae:.4f}, MSE: {mse:.4f}')
from plot_helper import plot_data
plot_data(X_test, y_test, mean, title='GPyTorch GPU Gaussian Process Predictions vs Ground Truth')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense  

#Define NN model
model_dnnstd = Sequential() 
model_dnnstd.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],) )) #21 Inputs as Dense layer
model_dnnstd.add(Dense(1,activation='linear')) # linear Output layer

model_dnnstd.compile(optimizer='adam', loss='mse', metrics=['mae'])

#Train the NN model and save training history
history_dnnstd=model_dnnstd.fit(
          X_train,
          y_train,
          epochs=100,
          batch_size=32)



In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, regularizers

#Define DNN model and use "diamond" shape 512-256-128-64-64-1
model_dnnopt = models.Sequential()
model_dnnopt.add(Dense(512, activation='relu', input_shape=(21,),kernel_regularizer=regularizers.l2(1e-4))) #21 Inputs and Dense layer already imported above
model_dnnopt.add(layers.BatchNormalization()) #Batch Normalization layer


model_dnnopt.add(Dense(256, activation='relu')) #256 Neurons
model_dnnopt.add(layers.Dropout(0.3))         #Dropout layer to reduce overfitting
model_dnnopt.add(Dense(128, activation='relu')) # 128 Neurons
model_dnnopt.add(layers.BatchNormalization()) 

model_dnnopt.add(Dense(64, activation='relu'))
model_dnnopt.add(Dense(64, activation='relu'))
model_dnnopt.add(Dense(1,activation='linear')) #Linear output layer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) 

model_dnnopt.compile(optimizer=optimizer, loss=tf.keras.losses.Huber(), metrics=['mae','mse'])




In [ ]:
#Define learning rate scheduler and early stopping
lr_scheduler = callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2,    # Cut LR by 80% when progress stalls
    patience=5,    # Wait 5 epochs before reducing LR
    min_lr=1e-7,   # Set a minimum learning rate
    verbose=1      # Print Message when LR is reduced
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss for early stopping
    patience=12,         # Number of epochs with no improvement before stopping
    restore_best_weights=True # Restore model weights from the epoch with the best value
)


history_dnnopt = model_dnnopt.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=200,          # Increased epochs to allow LR scheduler to work
    batch_size=32,       
    callbacks=[early_stopping, lr_scheduler],
   )


In [ ]:
#Eval linear regression

from plot_helper import evaluate_model, plot_data
evaluate_model(model_lr, X_test, y_test)
plot_data(X_test, y_test, model_lr.predict(X_test), title="Model Predictions vs Ground Truth on Test Data")

In [ ]:
#Eval RFRegressor

from plot_helper import evaluate_model, plot_data
evaluate_model(model_rfreg, X_test, y_test)
plot_data(X_test, y_test, model_rfreg.predict(X_test), title="Model Predictions vs Ground Truth on Test Data")

In [ ]:
#Eval DNN standard

from plot_helper import evaluate_model, plot_data
evaluate_model(model_dnnstd, X_test, y_test)
plot_data(X_test, y_test, model_dnnstd.predict(X_test), title="Model Predictions vs Ground Truth on Test Data")
plot_loss(history_dnnstd)


In [ ]:
#Eval DNN optimized

from plot_helper import evaluate_model, plot_data
evaluate_model(model_dnnopt, X_test, y_test)
plot_data(X_test, y_test, model_dnnopt.predict(X_test), title="Model Predictions vs Ground Truth on Test Data")
plot_loss(history_dnnopt)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error

names = ['Linear Regression', 'Random Forest Regressor', 'DNN Optimized', 'DNN Standard', 'GPyTorch GPU GP']
models_map = {
    'Linear Regression': globals().get('model_lr'),
    'Random Forest Regressor': globals().get('model_rfreg'),
    'DNN Optimized': globals().get('model_dnnopt'),
    'DNN Standard': globals().get('model_dnnstd'),
    'GPyTorch GPU GP': globals().get('model_gt'),
}
r2_list = []
rmse_list = []
for n in names:
    if n == 'GPyTorch GPU GP':  #Pytorch model needs special handling
        with torch.no_grad(): 
            preds = likelihood(model_gt(X_test_t)) #equal to scikit learn .predict()
            preds = preds.mean.cpu().numpy() #convert to numpy on CPU
    else:
        mdl = models_map.get(n)
        preds = mdl.predict(X_test)
        preds = np.asarray(preds).reshape(-1)

    r2 = r2_score(y_test, preds)
    mse = mean_squared_error(y_test, preds)
    rmse = np.sqrt(mse)
    r2_list.append(r2)
    rmse_list.append(rmse)

r2s = np.array(r2_list, dtype=float)
rmses = np.array(rmse_list, dtype=float)


colors = ['#4C72B0', '#DD8452', '#55A868', '#C44E52', '#8172B2']

# Create wide figure with two subplots side-by-side
fig = plt.figure(figsize=(14, 4.5))
fig.suptitle('Model Performance: R² and RMSE', fontsize=22, fontweight='bold')

# Left subplot: R²
ax1 = fig.add_subplot(1, 2, 1)
x = np.arange(len(names))
bars1 = ax1.bar(x, r2s, color=colors, edgecolor='black', linewidth=0.7)
ax1.set_xticks(x)
ax1.set_xticklabels(names, rotation=25, ha='right', fontsize=10) #Rotate x-axis labels for better readability
ax1.set_ylabel('R²', fontsize=12)
ax1.set_ylim(0.0, 1.0)
ax1.set_title('R² Score (higher is better)', fontsize=14)
ax1.yaxis.grid(True, linestyle='--', linewidth=0.6, alpha=0.6) 
plt.axhline(y=0.50, color='r', linestyle='-', label='50% R2') #Add horizontal line at R² = 0.50

# Add numeric labels above each R² bar
for rect, val in zip(bars1, r2s):
    height = rect.get_height()
    ax1.text(rect.get_x() + rect.get_width() / 2, height + 0.02, f'{val:.4f}', ha='center', va='bottom', fontsize=10) #.4f for 4 decimal places

# Right subplot: RMSE
ax2 = fig.add_subplot(1, 2, 2)
bars2 = ax2.bar(x, rmses, color=colors, edgecolor='black', linewidth=0.7)
ax2.set_xticks(x)
ax2.set_xticklabels(names, rotation=25, ha='right', fontsize=10) #Rotate x-axis labels for better readability
ax2.set_ylabel('RMSE', fontsize=12)
ax2.set_title('RMSE (lower is better)', fontsize=14)
ax2.yaxis.grid(True, linestyle='--', linewidth=0.6, alpha=0.6)

# Add numeric labels above each RMSE bar
for rect, val in zip(bars2, rmses):
    height = rect.get_height()
    ax2.text(rect.get_x() + rect.get_width() / 2, height + 0.0022, f'{val:.4f}', ha='center', va='bottom', fontsize=10) #.4f for 4 decimal places

# Tight layout and show
plt.tight_layout(rect=[0, 0, 1, 0.93])
plt.show()
